In [1]:
import cv2
import numpy as np

# https://stackoverflow.com/questions/10196198/how-to-remove-convexity-defects-in-a-sudoku-square

# 1. Image PreProcessing ( blur )
img = cv2.imread('sudoku.jpeg')
# img = cv2.imread('crossword1.png')
imgRaw = img.copy()

winName="raw image"
cv2.namedWindow(winName)
cv2.imshow(winName, imgRaw)
cv2.moveWindow(winName, 100, 100)

img = cv2.GaussianBlur(imgRaw, (5, 5), 0)

winName="blurred"
cv2.namedWindow(winName)
cv2.imshow(winName, img)
cv2.moveWindow(winName, 100, 150)

# 1.1 Image PreProcessing ( convert to gray )
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

winName="gray"
cv2.namedWindow(winName)
cv2.imshow(winName, gray)
cv2.moveWindow(winName, 100, 200)

# 1.2 Image PreProcessing ( Morphological closing operation )
mask = np.zeros((gray.shape), np.uint8)
kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
close = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, kernel1)
div = np.float32(gray)/(close)
res = np.uint8(cv2.normalize(div, div, 0, 255, cv2.NORM_MINMAX))

# need the rgb image for later 
res2 = cv2.cvtColor(res, cv2.COLOR_GRAY2BGR)

winName="res2"
cv2.namedWindow(winName)
cv2.imshow(winName, res2)
cv2.moveWindow(winName, 100, 250)

# 2. Finding Sudoku Square and Creating Mask Image
thresh = cv2.adaptiveThreshold( 
    res,
    maxValue=255.0,
    adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    thresholdType=cv2.THRESH_BINARY_INV,
    blockSize=19,
    C=9
)

winName="thresh"
cv2.namedWindow(winName)
cv2.imshow(winName, thresh)
cv2.moveWindow(winName, 100, 300)

(cnts, _) = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# find the biggest contour
max_area = 0
best_cnt = None
for cnt in cnts:
    area = cv2.contourArea(cnt)
    if area > 50:
        if area > max_area:
            max_area = area
            best_cnt = cnt

cv2.drawContours(mask, [best_cnt], 0, 255, -1)
cv2.drawContours(mask, [best_cnt], 0, 0, 2)

res = cv2.bitwise_and(res, mask)

winName="puzzle only"
cv2.namedWindow(winName)
cv2.imshow(winName, res)
cv2.moveWindow(winName, 100, 350)

# 3. Finding Vertical lines
kernel1X = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 10))

dx = cv2.Sobel(res, cv2.CV_16S, 1, 0)
dx = cv2.convertScaleAbs(dx)
cv2.normalize(dx, dx, 0, 255, cv2.NORM_MINMAX)
ret, close = cv2.threshold(dx, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
close = cv2.morphologyEx(close, cv2.MORPH_DILATE, kernel1X, iterations = 1)

(cnts, _) = cv2.findContours(close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
for cnt in cnts:
    x, y, w, h = cv2.boundingRect(cnt)
    if h/w > 5:
        cv2.drawContours(close, [cnt], 0, 255,-1)
    else:
        cv2.drawContours(close, [cnt], 0, 0,-1)
close = cv2.morphologyEx(close, cv2.MORPH_CLOSE, None, iterations = 2)
closeX = close.copy()

winName="vertical lines"
cv2.namedWindow(winName)
cv2.imshow(winName, closeX)
cv2.moveWindow(winName, 100, 400)

#4. Finding Horizontal Lines
kernel1Y = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 2))
dy = cv2.Sobel(res, cv2.CV_16S, 0, 2)
dy = cv2.convertScaleAbs(dy)
cv2.normalize(dy, dy, 0, 255, cv2.NORM_MINMAX)
ret, close = cv2.threshold(dy, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
close = cv2.morphologyEx(close, cv2.MORPH_DILATE, kernel1Y)

(cnts, _) = cv2.findContours(close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for cnt in cnts:
    x, y, w, h = cv2.boundingRect(cnt)
    if w/h > 5:
        cv2.drawContours(close, [cnt], 0, 255,-1)
    else:
        cv2.drawContours(close, [cnt], 0, 0,-1)

close = cv2.morphologyEx(close, cv2.MORPH_DILATE, None, iterations = 2)
closeY = close.copy()

winName="horizontal lines"
cv2.namedWindow(winName)
cv2.imshow(winName, closeY)
cv2.moveWindow(winName, 100, 450)

# 5. Finding Grid Points ( intersection of these two gives dots )
res = cv2.bitwise_and(closeX, closeY)

winName="intersections"
cv2.namedWindow(winName)
cv2.imshow(winName, res)
cv2.moveWindow(winName, 100, 500)

# 6. Correcting the defects
# find centroids and sort
(cnts, _) = cv2.findContours(res, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
centroids = []
for cnt in cnts:
    mom = cv2.moments(cnt)
    (x, y) = int(mom['m10']/mom['m00']), int(mom['m01']/mom['m00'])
    cv2.circle(img, (x, y), 4, (0, 255, 0),-1)
    centroids.append((x, y))

winName="Centroids"
cv2.namedWindow(winName)
cv2.imshow(winName, img)
cv2.moveWindow(winName, 100, 550)

# text green
textColor=(255, 0, 0)

# points blue
pointColor=(0, 255, 0)

labeled=res2.copy()
for index, pt in enumerate(centroids):
    cv2.putText(labeled, str(index), (int(pt[0]), int(pt[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, textColor, 2)
    cv2.circle(labeled, (int(pt[0]), int(pt[1])), 5, pointColor, -1)

winName="labeled"
cv2.namedWindow(winName)
cv2.imshow(winName, labeled)
cv2.moveWindow(winName, 100, 600)

# sorting
centroids = np.array(centroids, dtype = np.float32)
# print (len(centroids))
c = centroids.reshape((100, 2))
c2 = c[np.argsort(c[:, 1])]

b = np.vstack([c2[i*10:(i+1)*10][np.argsort(c2[i*10:(i+1)*10, 0])] for i in range(10)])
bm = b.reshape((10, 10, 2))

labeled_in_order=res2.copy()
for index, pt in enumerate(b):
    cv2.putText(labeled_in_order, str(index), (int(pt[0]), int(pt[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, textColor, 2)
    cv2.circle(labeled_in_order, (int(pt[0]), int(pt[1])), 5, pointColor, -1)

winName="labeled in order"
cv2.namedWindow(winName)
cv2.imshow(winName, labeled_in_order)
cv2.moveWindow(winName, 100, 650)

# create final using a mesh
output = np.zeros((450, 450, 3), np.uint8)
for i, j in enumerate(b):
    ri = int(i/10) # row index
    ci = i%10 # column index
    if ci != 9 and ri!=9:
        src = bm[ri:ri+2, ci:ci+2 , :].reshape((4, 2))
        dst = np.array( [ [ci*50, ri*50], [(ci+1)*50-1, ri*50], [ci*50, (ri+1)*50-1], [(ci+1)*50-1, (ri+1)*50-1] ], np.float32)
        retval = cv2.getPerspectiveTransform(src, dst)
        warp = cv2.warpPerspective(res2, retval, (450, 450))
        output[ri*50:(ri+1)*50-1 , ci*50:(ci+1)*50-1] = warp[ri*50:(ri+1)*50-1 , ci*50:(ci+1)*50-1].copy()

# This only warps the perspective using the outer 4 points, not a mesh
# src =  np.array( [bm[0][0], bm[0][9], bm[9][0], bm[9][9]], np.float32)
# dst = np.array( [[0, 0], [450, 0], [0, 450], [450, 450]], np.float32)
# retval = cv2.getPerspectiveTransform(src, dst)
# warp = cv2.warpPerspective(res2, retval, (450, 450))  
# output=warp.copy()

winName="final"
cv2.namedWindow(winName)
cv2.imshow(winName, output)
cv2.moveWindow(winName, 600, 100)

cv2.waitKey(0)
cv2.destroyAllWindows()